<a href="https://colab.research.google.com/github/reiniscimurs/gnn_with_pytorch/blob/main/section_5/01_gat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAT Implementation
Graph Attention Networks (GAT) Implementation
Train the model using a dataset with multiple graphs and mini-batch learning.
Utilize GPU for training by selecting "GPU" in "Edit" -> "Notebook Settings" -> "Hardware Accelerator".


## Install PyTorch Geometric
Install the library "PyTorch Geometric" for Graph Neural Networks (GNN) along with related libraries.

In [ ]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

## Load Dataset
Load the dataset "MUTAG" containing 188 graphs from TUDataset.


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG")

dataset = dataset.shuffle()  # Shuffle the dataset
dataset_train = dataset[:140]  # Training dataset
dataset_test = dataset[140:]  # Test dataset

batch_size = 64  # Batch size
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Model Construction
To implement the GAT layer, use GATConv() to build a simpler GAT model compared to the original paper.


```
GATConv(input_feature_dimensions, output_feature_dimensions, number_of_multi-head_attentions)
```  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GATConv  
Stack two GATConv layers and introduce "dropout," which randomly removes neurons, before the fully connected layer.


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

n_h = 64  # Number of features in the hidden layer
n_head = 32

class GAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.gat1 = GATConv(dataset.num_node_features,
                            n_h,
                            heads=n_head)
        self.gat2 = GATConv(n_h * n_head,
                            n_h,
                            heads=n_head)
        self.fc = nn.Linear(n_h * n_head, dataset.num_classes)  # Fully connected layer

        self.relu = nn.ReLU()  # ReLU
        self.dropout = nn.Dropout(p=0.5)  # Dropout: (p=dropout rate)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        x = self.gat1(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        x = self.relu(x)

        # Take the mean of each feature across all nodes
        x = global_mean_pool(x, batch)  # Convert to (batch_size, number of features)

        x = self.dropout(x)
        x = self.fc(x)

        return x

net = GAT()
net.cuda()  # GPU support

## Training

Prepare a function for model evaluation.



In [ ]:
def evaluate(loader):
    correct = 0  # Number of correct predictions

    for data in loader:
        data = data.cuda()  # GPU support
        out = net(data)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())

    return correct / len(loader.dataset)  # Accuracy


Train the model using the training data.

In [ ]:
from torch import optim

# Cross-entropy loss function
loss_fnc = nn.CrossEntropyLoss()

# Optimization algorithm
optimizer = optim.Adam(net.parameters())

for epoch in range(200):
    # Training
    net.train()  # Set to training mode
    for data in loader_train:
        data = data.cuda()  # GPU support

        optimizer.zero_grad()  # Step ①: Initialize gradients
        out = net(data)  # Step ②: Obtain predictions through forward pass
        loss = loss_fnc(out, data.y)  # Step ③: Calculate loss from predictions and ground truth

        loss.backward()  # Step ④: Backpropagate gradients to calculate gradients
        optimizer.step()  # Step ⑤: Update parameters using the optimization algorithm

    # Evaluation
    net.eval()  # Set to evaluation mode
    acc_train = evaluate(loader_train)
    acc_test = evaluate(loader_test)
    print("Epoch:", epoch,
          "acc_train:", str(acc_train*100) + "%",
          "acc_test:", str(acc_test*100) + "%")


## Model Evaluation
Evaluate the trained model.


In [ ]:
net.eval()  # Set to evaluation mode
acc_test = evaluate(loader_test)
print("Accuracy:", str(acc_test*100) + "%")